In [ ]:
import time
import numpy as np

In [4]:
def train_epoch(model, loader, optimizer,criterion, device='cpu'):
    """
      Trains a neural network model for one epoch using the specified data loader and optimizer.

      Args:
          model (nn.Module): The neural network model to be trained.
          loader (DataLoader): The PyTorch Geometric DataLoader containing the training data.
          optimizer (torch.optim.Optimizer): The PyTorch optimizer used for training the model.
          device (str): The device used for training the model (default: 'cpu').

      Returns:
          float: The mean loss value over all the batches in the DataLoader.

      """
    losses = []
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        targets = batch.y
        loss = criterion(out, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

In [2]:
def evaluate_epoch(model, loader, criterion, device='cpu'):
    """
    Evaluates the performance of a trained neural network model on a dataset using the specified data loader.

    Args:
        model (nn.Module): The trained neural network model to be evaluated.
        loader (DataLoader): The PyTorch Geometric DataLoader containing the evaluation data.
        device (str): The device used for evaluating the model (default: 'cpu').

    Returns:
        float: The mean loss value over all the batches in the DataLoader.

    """
    losses = []
    for batch in loader:
        model.eval()
        out = model(batch)
        targets = batch.y
        loss = criterion(out, targets)
        losses.append(loss.item())
    return np.mean(losses)

In [5]:
def train(num_epochs, model, train_loader, val_loader, optimizer, criterion, device='cpu', patience=5 ):
    # create vectors for the training and validation loss
    train_losses = []
    val_losses = []

    #start measuring time
    start_time = time.time()

    for epoch in range(1, num_epochs+1):
        # Model training
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device=device)

        # Model validation
        val_loss = evaluate_epoch(model, val_loader, criterion, device=device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Early stopping
        try:
            if val_losses[-1]>=val_losses[-2]:
                early_stop += 1
                if early_stop == patience:
                    print("Early stopping! Epoch:", epoch)
                    break
            else:
                early_stop = 0
        except:
            early_stop = 0

        if epoch%10 == 0:
            print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                  "\t validation loss:", np.round(val_loss,4))

    elapsed_time = time.time() - start_time
    print(f'Model training took {elapsed_time:.3f} seconds')
    return train_losses, val_losses